# TASK 1

In [1]:
import random
from heapq import nsmallest

grid = [
    ['S', '.', '.', '.', '.'],
    ['.', '#', '#', '.', '.'],
    ['.', '.', '.', '.', '.'],
    ['.', '#', '#', '.', 'T'],
    ['.', '.', '.', '.', '.']
]


In [2]:
start = (0, 0)
target = (3, 4)
k = 2  

In [3]:
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [4]:
def get_neighbors(pos):
    x, y = pos
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    neighbors = []
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 5 and 0 <= ny < 5 and grid[nx][ny] != '#':
            neighbors.append((nx, ny))
    return neighbors

In [5]:
def local_beam_search(start, target, k):
    current_positions = random.sample(get_neighbors(start), k)
    path = {pos: [start, pos] for pos in current_positions}
    nodes_expanded = 0
    iterations = 0
    
    while True:
        iterations += 1
        all_successors = []
        for pos in current_positions:
            successors = get_neighbors(pos)
            nodes_expanded += len(successors)
            for succ in successors:
                if succ not in path:
                    path[succ] = path[pos] + [succ]
                all_successors.append((succ, manhattan_distance(succ, target)))
        
        if not all_successors:
            return "Failed to reach target", nodes_expanded, iterations
        
        best_k = nsmallest(k, all_successors, key=lambda x: x[1])
        current_positions = [pos for pos, _ in best_k]
        
        if target in current_positions:
            return path[target], nodes_expanded, iterations


In [6]:
path,nodes_expanded, iterations = local_beam_search(start, target, k)
print("Path:", path)
print("Nodes Expanded(Space used):", nodes_expanded)
print("Iterations(Time):", iterations)

Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4)]
Nodes Expanded(Space used): 31
Iterations(Time): 6


# TASK 2

In [7]:
import time
from collections import deque
import copy

N = 9  

# 0 represents empty cells
sudoku_grid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]



In [8]:
def is_valid(grid, row, col, num):
    for i in range(N):
        if grid[row][i] == num or grid[i][col] == num:
            return False
    sub_x, sub_y = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if grid[sub_x + i][sub_y + j] == num:
                return False
    return True



In [9]:
def find_empty_cell(grid):
    
    for i in range(N):
        for j in range(N):
            if grid[i][j] == 0:
                return i, j
    return None



In [10]:
def print_grid(grid):
    for row in grid:
        print(" ".join(str(num) if num != 0 else '.' for num in row))
    print()



In [11]:
def backtracking_solve(grid, nodes_expanded=0, iterations=0):
    
    start_time = time.time()
    iterations += 1
    empty = find_empty_cell(grid)
    if not empty:
        return grid, time.time() - start_time, nodes_expanded, iterations
    row, col = empty
    
    for num in range(1, 10):
        nodes_expanded += 1
        if is_valid(grid, row, col, num):
            grid[row][col] = num
            result, exec_time, nodes_expanded, iterations = backtracking_solve(grid, nodes_expanded, iterations)
            if result:
                return result, exec_time, nodes_expanded, iterations
            grid[row][col] = 0  
    return None, time.time() - start_time, nodes_expanded, iterations

In [12]:
def forward_checking_solve(grid, nodes_expanded=0, iterations=0):
    def forward_check(grid, row, col, num):
        return is_valid(grid, row, col, num)
    
    start_time = time.time()
    iterations += 1
    empty = find_empty_cell(grid)
    if not empty:
        return grid, time.time() - start_time, nodes_expanded, iterations
    row, col = empty
    
    for num in range(1, 10):
        nodes_expanded += 1
        if forward_check(grid, row, col, num):
            grid[row][col] = num
            result, exec_time, nodes_expanded, iterations = forward_checking_solve(grid, nodes_expanded, iterations)
            if result:
                return result, exec_time, nodes_expanded, iterations
            grid[row][col] = 0
    return None, time.time() - start_time, nodes_expanded, iterations



In [13]:
def ac3(grid):
    queue = deque()
    domains = { (r, c): set(range(1, 10)) for r in range(N) for c in range(N) if grid[r][c] == 0 }
    
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                queue.append((r, c))
    
    while queue:
        row, col = queue.popleft()
        for num in list(domains[(row, col)]):
            if not is_valid(grid, row, col, num):
                domains[(row, col)].remove(num)
            if not domains[(row, col)]:
                return False, domains
    return True, domains



In [14]:
solution_bt, time_bt, nodes_bt, iter_bt = backtracking_solve(copy.deepcopy(sudoku_grid))
print("Backtracking Solution:")
print_grid(solution_bt)
print("Nodes Expanded:", nodes_bt)
print("Iterations:", iter_bt)

Backtracking Solution:
1 2 3 4 5 6 7 8 9
4 5 6 7 8 9 1 2 3
7 8 9 1 2 3 4 5 6
2 1 4 3 6 5 8 9 7
3 6 5 8 9 7 2 1 4
8 9 7 2 1 4 3 6 5
5 3 1 6 4 2 9 7 8
6 4 2 9 7 8 5 3 1
9 7 8 5 3 1 6 4 2

Nodes Expanded: 3195
Iterations: 392


In [15]:
solution_fc, time_fc, nodes_fc, iter_fc = forward_checking_solve(copy.deepcopy(sudoku_grid))
print("\nForward Checking Solution:")
print_grid(solution_fc)
print("Nodes Expanded:", nodes_fc)
print("Iterations:", iter_fc)


Forward Checking Solution:
1 2 3 4 5 6 7 8 9
4 5 6 7 8 9 1 2 3
7 8 9 1 2 3 4 5 6
2 1 4 3 6 5 8 9 7
3 6 5 8 9 7 2 1 4
8 9 7 2 1 4 3 6 5
5 3 1 6 4 2 9 7 8
6 4 2 9 7 8 5 3 1
9 7 8 5 3 1 6 4 2

Nodes Expanded: 3195
Iterations: 392


In [16]:
is_ac_consistent, domains = ac3(copy.deepcopy(sudoku_grid))
print("\nAC3 Consistency Check:", "Consistent" if is_ac_consistent else "Not Consistent")


AC3 Consistency Check: Consistent
